## DATA PREPARATION
In this notebook we will prepare the data in order to compare the clustering techniques. First we try to understand how the file us_equities_logreturns.parquet has been generated starting from raw data (us_equities.csv.gz) in order to be sure that we are fine with the filtering/transformations that were performed and use directly that file to perform our clastering techinques and compare different strategies.

In [4]:
#imports
import random
import numpy as np
import pandas as pd

In [26]:
# Import the two files
DF=pd.read_csv("us_equities.csv.gz",compression = 'gzip')
DF = DF.iloc[: , 1:]
display(DF)
DF2=pd.read_parquet("us_equities_logreturns.parquet")
display(DF2)

,^GSPC,GE,IBM,DIS,BA,CAT,AA,HPQ,DD,KO,...,CTX,CTR,DYN,AIB,PCU,KIM,SFN,AMR,TCO,S
0,16.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16286,1868.20,25.76,186.22,81.38,124.43,96.82,12.17,29.59,66.43,38.47,...,25.35,21.27,24.39,23.17,NaN,21.45,26.06,NaN,22.96,8.78
16287,1846.34,25.34,183.90,79.93,121.89,95.61,11.86,28.91,65.71,37.97,...,25.53,21.13,24.18,23.14,NaN,21.49,26.10,NaN,22.95,8.62
16288,1841.13,25.11,182.21,80.07,123.11,95.39,11.84,29.07,65.77,38.17,...,25.71,21.20,24.38,23.21,NaN,21.52,26.24,NaN,22.60,8.78
16289,1858.83,25.43,185.81,81.39,125.42,95.67,11.90,29.48,66.38,38.27,...,25.81,21.37,24.44,23.82,NaN,21.53,26.50,NaN,22.61,8.77


,^GSPC,GE,IBM,DIS,BA,CAT,AA,HPQ,DD,KO,...,NSM,CLP,CTX,CTR,DYN,AIB,KIM,SFN,TCO,S
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.011340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.004737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.002949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.005872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16150,0.000305,-0.005420,-0.002896,0.004063,-0.009916,-0.000207,-0.001642,-0.010756,0.006343,-0.000520,...,0.042972,NaN,0.001974,-0.001879,0.011547,0.003892,0.002334,0.011191,0.009188,0.000000
16151,-0.011770,-0.016439,-0.012537,-0.017978,-0.020624,-0.012576,-0.025803,-0.023249,-0.010898,-0.013082,...,-0.011131,NaN,0.007076,-0.006604,-0.008647,-0.001296,0.001863,0.001534,-0.000436,-0.018391
16152,-0.002826,-0.009118,-0.009232,0.001750,0.009959,-0.002304,-0.001688,0.005519,0.000913,0.005253,...,0.047769,NaN,0.007026,0.003307,0.008237,0.003020,0.001395,0.005350,-0.015368,0.018391
16153,0.009568,0.012663,0.019565,0.016351,0.018590,0.002931,0.005055,0.014005,0.009232,0.002616,...,-0.017531,NaN,0.003882,0.007987,0.002458,0.025942,0.000465,0.009860,0.000442,-0.001140


We notice that the two dataframes have different number of columns (=stocks). By looking at those columns we notice that the number of NaN's is really high and that the limit imposed is really close to the number of rows in the dataframe. We conclude that the first filtering applied to the dataframe is to drop all the columns that have more than half of the value equal to NaN. We perform the same on our df and continue our analysis.

In [ ]:
# Let's look at the difference
diff = [stock for stock in list(DF.columns) if stock not in list(DF2.columns)]
print(diff)
diffdf = DF[diff]
print(diffdf.isnull().sum(axis = 0))
print(DF.shape[0]/2)

In [27]:
diffdf = DF[diff]

['BAX', 'TAP', 'WRB', 'AVY', 'ATO', 'AJG', 'AIT', 'B', 'BMI', 'AZZ', 'CAS', 'ARL', 'CBT', 'BCE', 'GAS', 'AEM', 'BEN', 'ABX', 'AM', 'BBY', 'PRX', 'BRC', 'AEG', 'SUN', 'SOV', 'SGR', 'LIZ', 'ABK', 'ID', 'TKS', 'BMT', 'TBL', 'SBG', 'SEL', 'PPD', 'TPP', 'ARA', 'CXR', 'WYE', 'PCU', 'AMR']
BAX     8143
TAP     8846
WRB     8846
AVY     8846
ATO     8846
AJG     8846
AIT     8846
B       8846
BMI     8846
AZZ     8846
CAS     8846
ARL     8846
CBT     8848
BCE     8852
GAS     8852
AEM     8853
BEN     8855
ABX     8955
AM      8995
BBY     9004
PRX     9014
BRC     9272
AEG     9360
SUN     9908
SOV    12591
SGR    12764
LIZ    12822
ABK    13108
ID     13280
TKS    13701
BMT    14402
TBL    14524
SBG    14681
SEL    14750
PPD    15340
TPP    15341
ARA    15633
CXR    15639
WYE    15711
PCU    15964
AMR    16019
dtype: int64
8145.5


In [28]:
# Let's drop columns with more than half of NaN values and look again at the difference
row_num = DF.shape[0]
#there is a small difference for some reason (-4 to compensate it)
nan_col = DF.isnull().sum(axis=0)>(row_num/2-4)
DF=DF.drop(columns=DF.columns[nan_col])

diff = [stock for stock in list(DF.columns) if stock not in list(DF2.columns)]
print(diff)
diffdf = DF[diff]


[]


In [30]:
display(DF)
display(DF2)

,^GSPC,GE,IBM,DIS,BA,CAT,AA,HPQ,DD,KO,...,EXC,UNP,EIX,LUV,FDX,PCG,R,CNW,MOT,CSX
0,16.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16286,1868.20,25.76,186.22,81.38,124.43,96.82,12.17,29.59,66.43,38.47,...,30.18,187.54,51.70,23.90,137.79,43.58,77.19,40.83,NaN,28.89
16287,1846.34,25.34,183.90,79.93,121.89,95.61,11.86,28.91,65.71,37.97,...,30.51,185.86,51.77,23.60,136.68,44.04,76.54,39.84,NaN,28.46
16288,1841.13,25.11,182.21,80.07,123.11,95.39,11.84,29.07,65.77,38.17,...,30.64,185.14,52.06,23.92,136.76,44.35,77.23,39.46,NaN,28.38
16289,1858.83,25.43,185.81,81.39,125.42,95.67,11.90,29.48,66.38,38.27,...,31.02,187.37,52.40,23.98,137.58,44.62,77.95,39.77,NaN,28.63


,^GSPC,GE,IBM,DIS,BA,CAT,AA,HPQ,DD,KO,...,NSM,CLP,CTX,CTR,DYN,AIB,KIM,SFN,TCO,S
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.011340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.004737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.002949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.005872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16150,0.000305,-0.005420,-0.002896,0.004063,-0.009916,-0.000207,-0.001642,-0.010756,0.006343,-0.000520,...,0.042972,NaN,0.001974,-0.001879,0.011547,0.003892,0.002334,0.011191,0.009188,0.000000
16151,-0.011770,-0.016439,-0.012537,-0.017978,-0.020624,-0.012576,-0.025803,-0.023249,-0.010898,-0.013082,...,-0.011131,NaN,0.007076,-0.006604,-0.008647,-0.001296,0.001863,0.001534,-0.000436,-0.018391
16152,-0.002826,-0.009118,-0.009232,0.001750,0.009959,-0.002304,-0.001688,0.005519,0.000913,0.005253,...,0.047769,NaN,0.007026,0.003307,0.008237,0.003020,0.001395,0.005350,-0.015368,0.018391
16153,0.009568,0.012663,0.019565,0.016351,0.018590,0.002931,0.005055,0.014005,0.009232,0.002616,...,-0.017531,NaN,0.003882,0.007987,0.002458,0.025942,0.000465,0.009860,0.000442,-0.001140


In [31]:
DF=pd.read_csv("us_equities.csv.gz",compression = 'gzip')
DF = DF.iloc[: , 1:]

In [45]:
nulls = (~DF.iloc[:int(DF.shape[0]/2) , :].isnull()).sum(axis = 0)
nulls = nulls[nulls==0]
print(list(nulls.index))

['ALK', 'CLX', 'AMD', 'ABT', 'TGT', 'DUK', 'APD', 'ASH', 'BDX', 'ADM', 'HES', 'BCR', 'PPG', 'GPC', 'SNE', 'HST', 'AFG', 'RDC', 'TY', 'NL', 'PKD', 'NEM', 'MUR', 'TSO', 'MAS', 'WHR', 'GIS', 'DNB', 'NUE', 'WMS', 'VZ', 'JPM', 'CP', 'COO', 'T', 'ABM', 'AP', 'AME', 'AFL', 'ADI', 'NWL', 'PNW', 'AXE', 'ADX', 'AIR', 'PC', 'ARW', 'EGN', 'PNM', 'HD', 'TAP', 'WRB', 'AVY', 'ATO', 'AJG', 'AIT', 'B', 'BMI', 'AZZ', 'CAS', 'ARL', 'NU', 'BLL', 'CAG', 'AIG', 'CBT', 'BCE', 'GAS', 'BK', 'CB', 'AEM', 'BGG', 'BKH', 'BEN', 'BRE', 'BOH', 'BMS', 'D', 'LNC', 'NI', 'TE', 'TXT', 'WFC', 'WEC', 'CR', 'WGO', 'BT', 'K', 'CMI', 'CVS', 'KMB', 'GWW', 'RAD', 'HAS', 'CCK', 'WWW', 'CMS', 'ABX', 'HL', 'FAC', 'JCI', 'PPL', 'AM', 'AOS', 'BBY', 'PRX', 'UN', 'Y', 'LOW', 'WAG', 'HSY', 'DOV', 'MDP', 'GCO', 'FUR', 'ETN', 'MWV', 'GAM', 'CPB', 'SHW', 'PH', 'IR', 'OKE', 'FMC', 'SWK', 'RRD', 'SNA', 'SPW', 'GCI', 'TKR', 'NC', 'ICB', 'LUB', 'VFC', 'PHM', 'SVU', 'MDR', 'MTS', 'JHI', 'MCI', 'CNA', 'NEU', 'UNS', 'NVE', 'XEL', 'HNZ', 'NVR', 

In [34]:
# #The advantage of fixing the seed: always give the same result
# for i in range(2):
#     tst = pick_n_from_k(DF,5, i,onlynames = True)
#     display(tst)

# for i in range(2):
#     tst = pick_n_from_k(DF,5, i)
#     display(tst)
#     print('Here is the risK:')
#     display(tst.std(axis=0))
    
#     display(tst)